In [1]:
import matplotlib.pyplot as plt
import math
import datetime as dt
import pandas as pd
import numpy as np
from pathlib import Path
import yaml
import requests
import os
import json


In [2]:
with open(Path("config.yaml"), 'r') as f: 
    config = yaml.safe_load(f)

In [3]:
bearer_token = config['token']
# search_url = "https://api.twitter.com/2/tweets/search/all"
search_url = "https://api.twitter.com/2/tweets/search/recent"

In [20]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, params, encoding = "ISO-8859-1"):
    response = requests.request("GET", search_url, headers=headers, params=params)
    print(response.apparent_encoding)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [21]:
s_ticker = "PETR4"
count = 10
start_time = None
end_time = None
params = {
    "query": fr"(\{s_ticker}) (lang:pt)",
    "max_results": str(count),
    "tweet.fields": "created_at,lang",
}

if start_time:
    # Assign from and to datetime parameters for the API
    params["start_time"] = start_time
if end_time:
    params["end_time"] = end_timea

In [23]:
json_response

{'data': [{'lang': 'pt',
   'text': 'Principais mudanças do mês de junho:\n\n- Saída parcial PETR4 (venda de 10% com 57% de lucro)\n- Saída parcial BBDC4 (venda de 30% com 49% de lucro)\n- Saída Total CVCB3 (venda de 100% com 101% de lucro)\n\n- Aumento posição TASA4\n- Aumento posição SAPR11',
   'id': '1410714934300876806',
   'created_at': '2021-07-01T21:40:24.000Z'},
  {'lang': 'pt',
   'text': 'RT @FilipeVillegas: 📊 Mais negociadas investidor PF 🐟🐡🐠 (Lote #Fracionário)\n\nTicker / Var(%) / N° de Ações\n\n#BBDC4 / -1,3% / 228K\n#PRIO3 /…',
   'id': '1410713143828004868',
   'created_at': '2021-07-01T21:33:17.000Z'},
  {'lang': 'pt',
   'text': 'RT @FilipeVillegas: 📊 Ações +Termadas #termos 🦈\n\n🧐 Apostando numa alta ou defendendo posição\n\nTicker / Var(%) / N°Ações\n\n#BRDT3 / +7,2% / 4…',
   'id': '1410713130393608196',
   'created_at': '2021-07-01T21:33:14.000Z'},
  {'lang': 'pt',
   'text': '👀🆙 Ações da BR Distribuidora fecharam com maior alta do índice Ibovespa nesta quinta-fe

In [22]:
headers = create_headers(bearer_token)
json_response = connect_to_endpoint(search_url, headers, params)
json_response

utf-8
200
{
    "data": [
        {
            "created_at": "2021-07-01T21:40:24.000Z",
            "id": "1410714934300876806",
            "lang": "pt",
            "text": "Principais mudan\u00e7as do m\u00eas de junho:\n\n- Sa\u00edda parcial PETR4 (venda de 10% com 57% de lucro)\n- Sa\u00edda parcial BBDC4 (venda de 30% com 49% de lucro)\n- Sa\u00edda Total CVCB3 (venda de 100% com 101% de lucro)\n\n- Aumento posi\u00e7\u00e3o TASA4\n- Aumento posi\u00e7\u00e3o SAPR11"
        },
        {
            "created_at": "2021-07-01T21:33:17.000Z",
            "id": "1410713143828004868",
            "lang": "pt",
            "text": "RT @FilipeVillegas: \ud83d\udcca Mais negociadas investidor PF \ud83d\udc1f\ud83d\udc21\ud83d\udc20 (Lote #Fracion\u00e1rio)\n\nTicker / Var(%) / N\u00b0 de A\u00e7\u00f5es\n\n#BBDC4 / -1,3% / 228K\n#PRIO3 /\u2026"
        },
        {
            "created_at": "2021-07-01T21:33:14.000Z",
            "id": "1410713130393608196",
            "lang": "pt",

In [26]:
from datetime import datetime 
def get_data(tweet):
    """Gets twitter data from API request"""
    if "+" in tweet["created_at"]:
        s_datetime = tweet["created_at"].split(" +")[0]
    else:
        s_datetime = datetime.strptime(tweet["created_at"].split(".")[0], "%Y-%m-%dT%H:%M:%S").strftime(
            "%Y-%m-%d %H:%M:%S"
        )

    if "full_text" in tweet.keys():
        s_text = tweet["full_text"]
    else:
        s_text = tweet["text"]

    data = {"created_at": s_datetime, "text": s_text}
    return data
    
def clean_tweet(tweet: str, s_ticker: str) -> str:
    """Cleans tweets to be fed to sentiment model"""
    whitespace = re.compile(r"\s+")
    web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
    ticker = re.compile(fr"(?i)@{s_ticker}(?=\b)")
    user = re.compile(r"(?i)@[a-z0-9_]+")

    tweet = whitespace.sub(" ", tweet)
    tweet = web_address.sub("", tweet)
    tweet = ticker.sub(s_ticker, tweet)
    tweet = user.sub("", tweet)

    return tweet

In [27]:
df_tweets = pd.DataFrame()
for tweet in json_response["data"]:
    row = get_data(tweet)
    df_tweets = df_tweets.append(row, ignore_index=True)

In [28]:
df_tweets

,created_at,text
0,2021-06-29 21:07:14,RT @unusual_whales: 🚨 BREAKING NEWS 🚨\n\n@Trad...
1,2021-06-29 21:07:09,RT @CannabisCricket: $ALF $ALFIW 22.50 HOD $25...
2,2021-06-29 21:07:07,RT @CannabisCricket: $ALF $ALFIW 22.50 HOD $25...
3,2021-06-29 21:07:06,RT @masked_investor: So since 002 has been liv...
4,2021-06-29 21:06:58,RT @GMEshortsqueeze: $GME I wonder if these ar...
5,2021-06-29 21:06:55,We got someone’s attention #AMC #GME https://t...
6,2021-06-29 21:06:50,RT @WSB_Degenerate: $CLOV\n\nWhen borrow fees ...
7,2021-06-29 21:06:44,RT @cvpayne: .@Trading212 there is some confus...
8,2021-06-29 21:06:39,$XELA short squeeze for the rest of the week!!...
9,2021-06-29 21:06:39,RT @CannabisCricket: $BSQR $MRIN $AUUD $ALF $X...
